# 유저 행동 이벤트 로그 생성

In [1]:
# ===================================================================
# 1️⃣ 필수 라이브러리 Import
# ===================================================================

import os
import uuid
import json
import random
import gc
from datetime import datetime, timedelta, timezone, date
from typing import List, Dict, Any, Optional
import warnings

# 데이터 처리
import numpy as np
import pandas as pd

# Dask (분산 처리)
import dask
import dask.dataframe as dd
import dask.bag as db
from dask.diagnostics import ProgressBar
from dask import delayed

# 시각화 (필요시)
import matplotlib.pyplot as plt
import seaborn as sns

# 설정
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Nanum Gothic'
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다.")

✅ 모든 라이브러리가 성공적으로 로드되었습니다.


## 1. DB 데이터 불러오기

In [2]:
# ===================================================================
# 2️⃣ 데이터 로딩 및 기본 설정
# ===================================================================

# 레시피 데이터 읽기
recipes_df = pd.read_parquet('data/output/total_recipes.parquet')

# 사용자 데이터 읽기
users_df = pd.read_parquet('data/output/user.parquet')

# 사용자 프로필 데이터 읽기
profiles_df = pd.read_parquet('data/output/user_profiles.parquet')

print(f"✅ 데이터 로딩 완료:")
print(f"   - 레시피: {len(recipes_df):,}개")
print(f"   - 사용자: {len(users_df):,}명")
print(f"   - 프로필: {len(profiles_df):,}개")

# Demographic Segment 분포 정의
DEMOGRAPHIC_DISTRIBUTION = {
    'FEMALE_20S': 0.142,    # 14.2%
    'FEMALE_30S': 0.207,    # 20.7%
    'FEMALE_40_PLUS': 0.356, # 35.6%
    'MALE_20S': 0.062,      # 6.2%
    'MALE_30S': 0.085,      # 8.5%
    'MALE_40_PLUS': 0.148   # 14.8%
}

# 행동 태그 정의
INTENSITY_PERSONAS = {
    'POWER_USER': {'ratio': 0.15, 'description': '파워_유저, 주 5회 이상 활동'},
    'ACTIVE_USER': {'ratio': 0.55, 'description': '활성_유저, 주 2-4회 활동'},
    'CASUAL_USER': {'ratio': 0.30, 'description': '캐주얼_유저, 주 1회 이하 활동'}
}

COOKING_STYLE_PERSONAS = {
    'DESSERT_FOCUSED': {'ratio': 0.20, 'description': '디저트_중심, 베이킹 디저트 제작 선호'},
    'HEALTHY_CONSCIOUS': {'ratio': 0.25, 'description': '건강식_지향, 다이어트 웰빙 요리 선호'},
    'COMFORT_FOOD': {'ratio': 0.25, 'description': '든든한_식사, 메인 요리 한 끼 식사 선호'},
    'QUICK_CONVENIENT': {'ratio': 0.20, 'description': '간편_요리, 시간절약 간단 요리 선호'},
    'DIVERSE_EXPLORER': {'ratio': 0.10, 'description': '다양한_탐험, 특별한 패턴 없이 다양하게 탐색'}
}

✅ 데이터 로딩 완료:
   - 레시피: 208,183개
   - 사용자: 30,000명
   - 프로필: 30,000개


In [3]:
# ===================================================================
# 3️⃣ 시뮬레이션 환경 설정 및 S-커브 성장 모델
# ===================================================================

import math

# 시뮬레이션 기간 (1년)
SIMULATION_START_DATE = datetime(2024, 8, 1)
SIMULATION_END_DATE = datetime(2025, 7, 23)

# 목표 DAU 및 MAU (원래 값으로 복귀)
TARGET_DAU = 2000  # 원래 값 유지
TARGET_MAU = 30000  # 원래 값 유지

# 전체 사용자 대비 활성 사용자 비율 (DAU/등록사용자)
ACTIVE_USER_RATIO = 0.40

# S-커브 성장 함수 정의 (원래 값으로 복귀)
def calculate_s_curve_dau(day_of_year, start_dau=100, end_dau=2000, inflection_point=0.45):
    """
    S-커브 형태의 DAU 성장을 계산하는 함수
    
    성장 단계:
    - 0-120일(~4개월): 안정화 단계 (100 → 200명)
    - 120-270일(4-9개월): 급성장 단계 (200 → 1,600명) 
    - 270-365일(9-12개월): 성숙 단계 (1,600 → 2,000명)
    """
    # 정규화된 시간 (0~1)
    t = day_of_year / 365.0
    
    # S-커브 파라미터 조정
    steepness = 12  # 성장 급격함 정도
    shift = inflection_point  # 변곡점 위치
    
    # Logistic 함수 (S-커브)
    s_value = 1 / (1 + math.exp(-steepness * (t - shift)))
    
    # 목표 DAU 범위로 스케일링
    dau = start_dau + (end_dau - start_dau) * s_value
    
    return int(dau)

# 이벤트 스키마 정의
EVENT_SCHEMA = {
    'view_page': {
        'required_props': ['page_type'],
        'optional_props': ['page_id', 'referrer_type'],
        'next_events': ['search_recipe', 'view_recipe_list', 'click_auth_button', 'view_ads']
    },
    'click_auth_button': {
        'required_props': ['type'],
        'optional_props': [],
        'next_events': ['auth_success']
    },
    'auth_success': {
        'required_props': ['method', 'type'],
        'optional_props': [],
        'next_events': ['view_page', 'search_recipe']
    },
    'search_recipe': {
        'required_props': ['search_type'],
        'optional_props': ['search_keyword', 'selected_filters', 'result_count'],
        'next_events': ['view_recipe_list', 'click_recipe_from_list']
    },
    'view_recipe_list': {
        'required_props': ['list_type'],
        'optional_props': ['displayed_recipe_ids'],
        'next_events': ['click_recipe_from_list', 'search_recipe']
    },
    'click_recipe_from_list': {
        'required_props': ['recipe_id'],
        'optional_props': ['rank'],
        'next_events': ['click_bookmark', 'click_like', 'create_comment']
    },
    'click_bookmark': {
        'required_props': ['recipe_id', 'action'],
        'optional_props': [],
        'next_events': ['view_page', 'click_recipe_from_list']
    },
    'click_like': {
        'required_props': ['recipe_id', 'action'],
        'optional_props': [],
        'next_events': ['view_page', 'click_recipe_from_list', 'create_comment']
    },
    'create_comment': {
        'required_props': ['recipe_id'],
        'optional_props': ['comment_length'],
        'next_events': ['view_page', 'click_recipe_from_list']
    },
    'create_recipe_success': {
        'required_props': ['recipe_id'],
        'optional_props': ['category', 'ingredient_count'],
        'next_events': ['view_page']
    },
    'view_ads': {
        'required_props': ['ad_id'],
        'optional_props': ['ad_type', 'position'],
        'next_events': ['click_ads', 'view_page']
    },
    'click_ads': {
        'required_props': ['ad_id'],
        'optional_props': ['ad_type', 'position', 'target_url'],
        'next_events': ['view_page']
    }
}

# 기타 설정
INPUT_DATA_PATH = 'data/output/'
OUTPUT_LOG_PATH = 'event_logs/'
AB_TEST_START_DATE = datetime(2024, 10, 1)
AB_TEST_END_DATE = datetime(2024, 12, 31)
AB_TEST_SCENARIO_CODE = 'BOOKMARK_RECOMMENDATION_TEST'
AB_TEST_CONTROL_CTR = 0.05  # 기본 클릭률
AB_TEST_TREATMENT_CTR = 0.08  # 개선된 클릭률

print("✅ 시뮬레이션 환경 설정 완료 (세션당 이벤트 수 증가 버전)")
print(f"📅 시뮬레이션 기간: {SIMULATION_START_DATE.strftime('%Y-%m-%d')} ~ {SIMULATION_END_DATE.strftime('%Y-%m-%d')}")
print(f"🎯 목표 DAU: {TARGET_DAU:,}명, MAU: {TARGET_MAU:,}명 (원래 값 유지)")
print(f"📊 활성 유저 비율: {ACTIVE_USER_RATIO:.1%}")
print(f"📁 출력 경로: {OUTPUT_LOG_PATH}")
print(f"📈 전략: 세션당 이벤트 수 증가로 약 10,000,000개 목표")

✅ 시뮬레이션 환경 설정 완료 (세션당 이벤트 수 증가 버전)
📅 시뮬레이션 기간: 2024-08-01 ~ 2025-07-23
🎯 목표 DAU: 2,000명, MAU: 30,000명 (원래 값 유지)
📊 활성 유저 비율: 40.0%
📁 출력 경로: event_logs/
📈 전략: 세션당 이벤트 수 증가로 약 10,000,000개 목표


In [4]:
# ===================================================================
# 4️⃣ 사용자 세그먼트 및 행동 태그 할당
# ===================================================================

def assign_user_personas(users_df, profiles_df):
    """사용자별 행동 태그 및 세그먼트 할당"""
    
    print("🎭 사용자 페르소나 할당 시작...")
    
    # 사용자와 프로필 병합
    merged_df = pd.merge(users_df, profiles_df, left_on='id', right_on='user_id', how='inner')
    
    # Demographic Segment 할당 (실제 분포에 맞게)
    segment_list = list(DEMOGRAPHIC_DISTRIBUTION.keys())
    segment_weights = list(DEMOGRAPHIC_DISTRIBUTION.values())
    
    merged_df['demographic_segment'] = np.random.choice(
        segment_list, 
        size=len(merged_df), 
        p=segment_weights
    )
    
    # 행동 태그 1: 서비스 이용 강도
    intensity_list = list(INTENSITY_PERSONAS.keys())
    intensity_weights = [persona['ratio'] for persona in INTENSITY_PERSONAS.values()]
    
    merged_df['intensity_persona'] = np.random.choice(
        intensity_list,
        size=len(merged_df),
        p=intensity_weights
    )
    
    # 행동 태그 2: 요리 스타일 선호도
    cooking_list = list(COOKING_STYLE_PERSONAS.keys())
    cooking_weights = [persona['ratio'] for persona in COOKING_STYLE_PERSONAS.values()]
    
    merged_df['cooking_style_persona'] = np.random.choice(
        cooking_list,
        size=len(merged_df),
        p=cooking_weights
    )
    
    print(f"✅ 페르소나 할당 완료: {len(merged_df):,}명")
    
    # 분포 확인
    print(f"\n📊 Demographic Segment 분포:")
    segment_dist = merged_df['demographic_segment'].value_counts(normalize=True)
    for segment, ratio in segment_dist.items():
        print(f"   - {segment}: {ratio:.1%}")
    
    print(f"\n⚡ 이용 강도 분포:")
    intensity_dist = merged_df['intensity_persona'].value_counts(normalize=True)
    for intensity, ratio in intensity_dist.items():
        print(f"   - {intensity}: {ratio:.1%}")
    
    print(f"\n🍳 요리 스타일 분포:")
    cooking_dist = merged_df['cooking_style_persona'].value_counts(normalize=True)
    for cooking, ratio in cooking_dist.items():
        print(f"   - {cooking}: {ratio:.1%}")
    
    return merged_df

print("✅ 사용자 페르소나 할당 함수 준비 완료")

✅ 사용자 페르소나 할당 함수 준비 완료


In [5]:
# ===================================================================
# 5️⃣ 이벤트 생성 핵심 함수들
# ===================================================================

def generate_event_properties(event_name, context, recipes_df):
    """이벤트별 속성 생성 (올바른 스키마에 맞게)"""
    
    properties = {}
    
    if event_name == 'view_page':
        pages = ['start', 'main', 'recipe_detail', 'profile', 'search_result']
        properties['page_name'] = context.get('page_name', random.choice(pages))
        
        if random.random() < 0.3:  # 30% 확률로 referrer 포함
            properties['referrer'] = random.choice(['https://google.com', 'https://naver.com', 'https://facebook.com', ''])
        
        if properties['page_name'] == 'recipe_detail' and context.get('recipe_id'):
            properties['path'] = f"/recipes/{context['recipe_id']}"
    
    elif event_name == 'click_auth_button':
        properties['type'] = random.choice(['signup', 'login'])
    
    elif event_name == 'auth_success':
        properties['method'] = random.choice(['email', 'kakao', 'google', 'naver'])
        properties['type'] = random.choice(['signup', 'login'])
    
    elif event_name == 'search_recipe':
        properties['search_type'] = random.choice(['category', 'ingredient', 'menu'])
        
        if random.random() < 0.7:  # 70% 확률로 검색어 포함
            keywords = ['치킨', '파스타', '샐러드', '스테이크', '케이크', '볶음밥', '국물요리']
            properties['search_keyword'] = random.choice(keywords)
        
        if random.random() < 0.4:  # 40% 확률로 필터 적용
            filters = ['cooking_time:30min', 'difficulty:easy', 'vegetarian:true']
            properties['selected_filters'] = random.sample(filters, random.randint(1, 2))
        
        properties['result_count'] = random.randint(5, 50)
    
    elif event_name == 'view_recipe_list':
        list_types = ['popular', 'recommended', 'search_result', 'trending']
        properties['list_type'] = random.choice(list_types)
        
        if random.random() < 0.6:  # 60% 확률로 표시된 레시피 ID 포함
            displayed_count = random.randint(5, 20)
            if not recipes_df.empty and 'id' in recipes_df.columns:
                recipe_sample = recipes_df.sample(n=min(displayed_count, len(recipes_df)))
                # String 타입으로 변환
                properties['displayed_recipe_ids'] = [str(x) for x in recipe_sample['id'].tolist()]
    
    elif event_name == 'click_recipe_from_list':
        if context and context.get('recipe_id'):
            properties['recipe_id'] = str(context['recipe_id']) if pd.notna(context['recipe_id']) else None
        elif not recipes_df.empty and 'id' in recipes_df.columns:
            recipe_id = recipes_df.sample(1)['id'].iloc[0]
            properties['recipe_id'] = str(recipe_id) if pd.notna(recipe_id) else None
        else:
            properties['recipe_id'] = f"recipe_{random.randint(1, 1000)}"
        
        if random.random() < 0.5:  # 50% 확률로 순위 포함
            properties['rank'] = random.randint(1, 20)
    
    elif event_name in ['click_bookmark', 'click_like']:
        if context and context.get('recipe_id'):
            properties['recipe_id'] = str(context['recipe_id']) if pd.notna(context['recipe_id']) else None
        elif not recipes_df.empty and 'id' in recipes_df.columns:
            recipe_id = recipes_df.sample(1)['id'].iloc[0]
            properties['recipe_id'] = str(recipe_id) if pd.notna(recipe_id) else None
        
        properties['action'] = random.choice(['add', 'remove']) if event_name == 'click_bookmark' else random.choice(['like', 'unlike'])
    
    elif event_name == 'create_comment':
        if context and context.get('recipe_id'):
            properties['recipe_id'] = str(context['recipe_id']) if pd.notna(context['recipe_id']) else None
        elif not recipes_df.empty and 'id' in recipes_df.columns:
            recipe_id = recipes_df.sample(1)['id'].iloc[0]
            properties['recipe_id'] = str(recipe_id) if pd.notna(recipe_id) else None
        
        if random.random() < 0.6:  # 60% 확률로 댓글 길이 포함
            properties['comment_length'] = random.randint(10, 200)
    
    elif event_name == 'create_recipe_success':
        if not recipes_df.empty and 'id' in recipes_df.columns:
            recipe_id = recipes_df.sample(1)['id'].iloc[0]
            properties['recipe_id'] = str(recipe_id) if pd.notna(recipe_id) else None
        
        if random.random() < 0.8:  # 80% 확률로 카테고리 포함
            categories = ['한식', '중식', '일식', '양식', '디저트', '음료']
            properties['category'] = random.choice(categories)
        
        if random.random() < 0.7:  # 70% 확률로 재료 수 포함
            properties['ingredient_count'] = random.randint(3, 15)
    
    elif event_name == 'view_ads':
        properties['ad_id'] = f"ad_{random.randint(1000, 9999)}"
        
        if random.random() < 0.8:  # 80% 확률로 광고 타입 포함
            properties['ad_type'] = random.choice(['banner', 'video', 'native'])
        
        if random.random() < 0.6:  # 60% 확률로 위치 포함
            properties['position'] = random.choice(['top', 'middle', 'bottom', 'sidebar'])
    
    elif event_name == 'click_ads':
        properties['ad_id'] = context.get('ad_id', f"ad_{random.randint(1000, 9999)}")
        
        if random.random() < 0.8:
            properties['ad_type'] = random.choice(['banner', 'video', 'native'])
        
        if random.random() < 0.6:
            properties['position'] = random.choice(['top', 'middle', 'bottom', 'sidebar'])
        
        if random.random() < 0.9:  # 90% 확률로 목적지 URL 포함
            properties['target_url'] = f"https://partner-site.com/promotion/{random.randint(1, 100)}"
    
    return properties

def generate_user_session_flow(user_data, session_time, recipes_df):
    """사용자별 세션 플로우 생성 (올바른 스키마 기반, 세션당 이벤트 수 대폭 증가 버전)"""
    
    session_id = str(uuid.uuid4())
    events = []
    current_time = session_time
    context = {}
    
    # 페르소나별 세션 길이 결정 (세션당 이벤트 수 2.24배 증가)
    intensity = user_data['intensity_persona']
    session_lengths = {
        'POWER_USER': random.randint(18, 34),   # 8-15 → 18-34 (약 2.24배)
        'ACTIVE_USER': random.randint(9, 18),  # 4-8 → 9-18 (약 2.24배)
        'CASUAL_USER': random.randint(4, 9)    # 2-4 → 4-9 (약 2.24배)
    }
    
    max_events = session_lengths.get(intensity, 5)
    
    # 세션 시작 이벤트 (항상 view_page 또는 auth_success로 시작)
    start_events = ['view_page', 'auth_success']
    current_event = random.choice(start_events)
    
    for _ in range(max_events):
        # 이벤트 속성 생성
        properties = generate_event_properties(current_event, context, recipes_df)
        
        # 현재 페이지 정보 설정
        page_name = properties.get('page_name', 'main')
        page_url = f"https://reciping.co.kr/{page_name}"
        page_path = f"/{page_name}"
        
        # context 객체 구성 (올바른 스키마에 맞게)
        context_obj = {
            "page": {
                "name": page_name,
                "url": page_url,
                "path": page_path
            },
            "user_segment": str(user_data['demographic_segment'])
        }
        
        # anonymous_id 생성 (빈 값이면 UUID 생성)
        anonymous_id = str(user_data.get('anonymous_id', ''))
        if not anonymous_id or anonymous_id == '':
            anonymous_id = str(uuid.uuid4())
        
        # 이벤트 기록 (올바른 스키마에 맞게)
        event = {
            'event_name': current_event,
            'event_id': str(uuid.uuid4()),
            'user_id': str(user_data['id']) if pd.notna(user_data['id']) else None,
            'anonymous_id': anonymous_id,
            'session_id': session_id,
            'context': json.dumps(context_obj, ensure_ascii=False),
            'event_properties': json.dumps(properties, default=str, ensure_ascii=False),
            'timestamp': current_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        }
        
        events.append(event)
        
        # 컨텍스트 업데이트 (안전한 타입 변환)
        if 'recipe_id' in properties and properties['recipe_id'] is not None:
            context['recipe_id'] = str(properties['recipe_id'])
        if 'ad_id' in properties:
            context['ad_id'] = str(properties['ad_id'])
        
        # 다음 이벤트 결정
        schema = EVENT_SCHEMA.get(current_event, {})
        next_events = schema.get('next_events', ['view_page'])
        
        if next_events and random.random() < 0.8:  # 80% 확률로 스키마 따름
            current_event = random.choice(next_events)
        else:
            current_event = random.choice(['view_page', 'search_recipe', 'view_recipe_list'])
        
        # 시간 증가 (5초 ~ 2분)
        current_time += timedelta(seconds=random.randint(5, 120))
    
    return events

print("✅ 이벤트 생성 핵심 함수들 준비 완료 (세션당 이벤트 수 2.24배 증가)")
print("📊 세션당 이벤트 수:")
print("   - POWER_USER: 8-15개 → 18-34개 (약 2.24배)")
print("   - ACTIVE_USER: 4-8개 → 9-18개 (약 2.24배)")
print("   - CASUAL_USER: 2-4개 → 4-9개 (약 2.24배)")

✅ 이벤트 생성 핵심 함수들 준비 완료 (세션당 이벤트 수 2.24배 증가)
📊 세션당 이벤트 수:
   - POWER_USER: 8-15개 → 18-34개 (약 2.24배)
   - ACTIVE_USER: 4-8개 → 9-18개 (약 2.24배)
   - CASUAL_USER: 2-4개 → 4-9개 (약 2.24배)

📊 세션당 이벤트 수:
   - POWER_USER: 8-15개 → 18-34개 (약 2.24배)
   - ACTIVE_USER: 4-8개 → 9-18개 (약 2.24배)
   - CASUAL_USER: 2-4개 → 4-9개 (약 2.24배)


In [6]:
# ===================================================================
# 6️⃣ 올바른 스키마 이벤트 로그 생성 함수
# ===================================================================

def get_realistic_session_time(target_date):
    """현실적인 시간대 분포를 가진 세션 시작 시간 생성"""
    
    # 시간대별 가중치 (레시피 검색 서비스 특성 반영)
    hourly_weights = {
        0: 0.5,   # 00시 - 매우 낮음 (심야)
        1: 0.3,   # 01시 - 매우 낮음
        2: 0.2,   # 02시 - 매우 낮음
        3: 0.2,   # 03시 - 매우 낮음
        4: 0.3,   # 04시 - 매우 낮음
        5: 0.8,   # 05시 - 낮음 (새벽 운동 후)
        6: 3.2,   # 06시 - 아침 준비 시작
        7: 5.8,   # 07시 - 아침식사 준비 피크
        8: 4.1,   # 08시 - 아침 마무리
        9: 2.3,   # 09시 - 오전 간식
        10: 1.8,  # 10시 - 브런치 준비
        11: 4.5,  # 11시 - 점심 준비 시작
        12: 6.2,  # 12시 - 점심시간 피크
        13: 3.9,  # 13시 - 점심 마무리
        14: 2.1,  # 14시 - 오후 간식
        15: 2.8,  # 15시 - 간식/디저트 시간
        16: 3.4,  # 16시 - 저녁 준비 고민 시작
        17: 7.8,  # 17시 - 퇴근 후 저녁 준비 시작 🔥
        18: 12.5, # 18시 - 저녁 준비 대피크 🔥🔥🔥
        19: 11.3, # 19시 - 저녁 요리 시간 🔥🔥
        20: 8.7,  # 20시 - 저녁 마무리 🔥
        21: 5.4,  # 21시 - 다음날 준비 or 야식
        22: 3.8,  # 22시 - 야식 레시피
        23: 2.1   # 23시 - 늦은 야식
    }
    
    # 요일별 가중치 (주말 vs 평일)
    weekday = target_date.weekday()  # 0=월요일, 6=일요일
    
    if weekday >= 5:  # 주말 (토, 일)
        # 주말에는 더 늦게 일어나고, 브런치 문화, 저녁도 조금 늦어짐
        weekend_multiplier = {
            0: 0.8, 1: 0.6, 2: 0.4, 3: 0.4, 4: 0.5, 5: 0.6,
            6: 0.7, 7: 0.8, 8: 1.2, 9: 1.8, 10: 2.5, 11: 2.8,  # 브런치 시간 증가
            12: 1.8, 13: 1.5, 14: 1.3, 15: 1.4, 16: 1.2,
            17: 1.1, 18: 1.3, 19: 1.4, 20: 1.2, 21: 1.1,  # 저녁 시간 분산
            22: 1.2, 23: 1.1  # 주말 야식 증가
        }
    else:  # 평일
        weekend_multiplier = {hour: 1.0 for hour in range(24)}
    
    # 최종 가중치 계산
    final_weights = []
    hours = []
    
    for hour in range(24):
        weight = hourly_weights[hour] * weekend_multiplier[hour]
        final_weights.append(weight)
        hours.append(hour)
    
    # 가중치 기반 시간 선택
    selected_hour = random.choices(hours, weights=final_weights)[0]
    selected_minute = random.randint(0, 59)
    selected_second = random.randint(0, 59)
    
    return target_date + timedelta(
        hours=selected_hour,
        minutes=selected_minute,
        seconds=selected_second
    )

def generate_correct_schema_event_logs():
    """올바른 스키마에 맞는 이벤트 로그 생성"""
    
    print("🚀 올바른 스키마로 이벤트 로그 생성 시작!")
    print("=" * 70)
    
    # Step 1: 디렉토리 생성
    os.makedirs(OUTPUT_LOG_PATH, exist_ok=True)
    
    # Step 2: 사용자 페르소나 할당
    print("👥 Step 1: 사용자 페르소나 할당...")
    profiles_with_personas = assign_user_personas(users_df, profiles_df)
    
    # Step 3: 데이터 준비
    print("📂 Step 2: 데이터 준비...")
    recipes_sample = recipes_df.sample(n=min(50000, len(recipes_df)), random_state=42)
    
    print(f"   사용자: {len(users_df):,}명")
    print(f"   레시피: {len(recipes_sample):,}개 (샘플링)")
    print(f"   프로필: {len(profiles_with_personas):,}개")
    
    # Step 4: 시뮬레이션 일정 생성
    print("\n📅 Step 3: 시뮬레이션 일정 생성...")
    
    current_date = SIMULATION_START_DATE
    date_info_list = []
    total_days = 0
    
    while current_date <= SIMULATION_END_DATE:
        day_of_year = (current_date - SIMULATION_START_DATE).days
        dau = calculate_s_curve_dau(day_of_year)
        
        date_info_list.append({
            'date': current_date,
            'dau': dau,
            'day_of_year': day_of_year
        })
        
        current_date += timedelta(days=1)
        total_days += 1
    
    print(f"   총 {total_days}일 시뮬레이션")
    print(f"   시작 DAU: {date_info_list[0]['dau']:,}명")
    print(f"   종료 DAU: {date_info_list[-1]['dau']:,}명")
    
    # Step 5: 사용자 배치 생성
    print("\n👥 Step 4: 사용자 배치 생성...")
    user_batches = []
    for month in range(12):
        batch_users = profiles_with_personas.sample(n=min(TARGET_MAU, len(profiles_with_personas)), 
                                               random_state=42 + month)
        user_batches.append(batch_users)
    
    print(f"   월별 사용자 배치: {len(user_batches)}개")
    
    # Step 6: 이벤트 생성 (월별 처리)
    print("\n🔄 Step 5: 이벤트 생성 시작...")
    print("🕐 현실적인 시간대 분포 적용 - 저녁 피크(17-20시), 점심(11-13시), 아침(6-9시)")
    
    all_events = []
    
    # 월별로 처리
    monthly_batches = []
    current_month_days = []
    current_month = SIMULATION_START_DATE.month
    
    for date_info in date_info_list:
        if date_info['date'].month == current_month:
            current_month_days.append(date_info)
        else:
            if current_month_days:
                monthly_batches.append(current_month_days)
            current_month_days = [date_info]
            current_month = date_info['date'].month
    
    if current_month_days:
        monthly_batches.append(current_month_days)
    
    # 월별 처리 실행
    for month_idx, month_days in enumerate(monthly_batches):
        print(f"\n   📆 {month_idx + 1}월 처리 중... ({len(month_days)}일)")
        
        # 해당 월의 사용자 배치 선택
        user_batch = user_batches[month_idx % len(user_batches)]
        
        month_events = []
        
        for day_info in month_days:
            target_date = day_info['date']
            target_dau = day_info['dau']
            
            # 해당 일의 활성 사용자 샘플링
            active_users = user_batch.sample(n=min(target_dau, len(user_batch)), random_state=42)
            
            for _, user_data in active_users.iterrows():
                # 사용자별 세션 수 결정 (원래 값으로 복귀)
                intensity = user_data['intensity_persona']
                session_counts = {
                    'POWER_USER': random.randint(2, 4),  # 원래 값
                    'ACTIVE_USER': random.randint(1, 2), # 원래 값
                    'CASUAL_USER': 1                     # 원래 값
                }
                
                num_sessions = session_counts.get(intensity, 1)
                
                for session_idx in range(num_sessions):
                    # 현실적인 시간대 분포를 적용한 세션 시작 시간 생성
                    session_start = get_realistic_session_time(target_date)
                    
                    # 세션 이벤트 생성 (세션당 이벤트 수는 증가됨)
                    session_events = generate_user_session_flow(user_data, session_start, recipes_sample)
                    month_events.extend(session_events)
        
        all_events.extend(month_events)
        print(f"      ✅ {len(month_events):,}개 이벤트 생성됨 (누적: {len(all_events):,}개)")
        
        # 중간 저장 (메모리 관리)
        if len(all_events) > 500000:  # 50만개마다 중간 저장
            print(f"      💾 중간 저장 중... ({len(all_events):,}개)")
            
            temp_df = pd.DataFrame(all_events)
            temp_file = os.path.join(OUTPUT_LOG_PATH, f'correct_temp_events_month_{month_idx + 1}.parquet')
            temp_df.to_parquet(temp_file, index=False)
            
            all_events = []
            del temp_df
            gc.collect()
    
    # Step 7: 최종 통합 및 저장
    print(f"\n💾 Step 6: 최종 데이터 통합 및 저장...")
    
    try:
        # 임시 파일들 확인
        temp_files = [f for f in os.listdir(OUTPUT_LOG_PATH) if f.startswith('correct_temp_events_')]
        
        if temp_files:
            print(f"   임시 파일 {len(temp_files)}개 통합 중...")
            
            all_dfs = []
            for temp_file in temp_files:
                temp_path = os.path.join(OUTPUT_LOG_PATH, temp_file)
                df = pd.read_parquet(temp_path)
                all_dfs.append(df)
            
            if all_events:
                final_df = pd.DataFrame(all_events)
                all_dfs.append(final_df)
            
            combined_df = pd.concat(all_dfs, ignore_index=True)
            
        else:
            combined_df = pd.DataFrame(all_events)
        
        # 최종 파일 저장
        output_file = os.path.join(OUTPUT_LOG_PATH, 'event_logs_10m.parquet')
        
        if os.path.exists(output_file):
            print(f"   기존 파일 삭제: {output_file}")
            os.remove(output_file)
        
        print(f"   최종 파일 저장 중: {output_file}")
        combined_df.to_parquet(output_file, index=False)
        
        # 임시 파일 정리
        for temp_file in temp_files:
            temp_path = os.path.join(OUTPUT_LOG_PATH, temp_file)
            if os.path.exists(temp_path):
                os.remove(temp_path)
        
        # 최종 통계
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
        
        print("=" * 70)
        print("🎉 올바른 스키마 이벤트 로그 생성 완료!")
        print("=" * 70)
        print(f"📁 저장 위치: {output_file}")
        print(f"📊 총 이벤트 수: {len(combined_df):,}개")
        print(f"💽 파일 크기: {file_size_mb:.2f} MB")
        print(f"📅 시뮬레이션 기간: {total_days}일")
        print(f"👥 고유 사용자: {combined_df['user_id'].nunique():,}명")
        print(f"🎯 이벤트 타입: {combined_df['event_name'].nunique()}개")
        print(f"🕐 현실적인 시간대 분포 적용됨!")
        
        # 스키마 검증
        print(f"\n🔍 올바른 스키마 검증:")
        required_columns = ['event_name', 'event_id', 'user_id', 'anonymous_id', 'session_id', 'context', 'event_properties', 'timestamp']
        
        missing_columns = [col for col in required_columns if col not in combined_df.columns]
        if missing_columns:
            print(f"❌ 누락된 컬럼: {missing_columns}")
        else:
            print("✅ 모든 필수 컬럼 존재")
        
        print(f"\n📋 생성된 컬럼:")
        for col in combined_df.columns:
            print(f"   - {col}: {combined_df[col].dtype}")
        
        return combined_df
        
    except Exception as e:
        print(f"   ❌ 최종 저장 오류: {e}")
        return None

print("✅ 올바른 스키마 이벤트 로그 생성 함수 준비 완료 (현실적인 시간대 분포 + 세션당 이벤트 수 증가)")
print("📊 전략: DAU/MAU 원래값 유지 + 세션당 이벤트 수 2.24배 증가 + 현실적인 시간대 분포")
print("🕐 시간대 특성:")
print("   - 🔥 저녁 피크: 17-20시 (퇴근 후 저녁 준비)")
print("   - 🍽️ 점심 피크: 11-13시 (점심 준비 시간)")  
print("   - 🌅 아침 시간: 6-9시 (아침식사 준비)")
print("   - 🌙 심야/새벽: 0-5시 (매우 낮은 활동)")
print("   - 📅 주말: 브런치 문화 반영, 저녁 시간 분산")

✅ 올바른 스키마 이벤트 로그 생성 함수 준비 완료 (현실적인 시간대 분포 + 세션당 이벤트 수 증가)
📊 전략: DAU/MAU 원래값 유지 + 세션당 이벤트 수 2.24배 증가 + 현실적인 시간대 분포
🕐 시간대 특성:
   - 🔥 저녁 피크: 17-20시 (퇴근 후 저녁 준비)
   - 🍽️ 점심 피크: 11-13시 (점심 준비 시간)
   - 🌅 아침 시간: 6-9시 (아침식사 준비)
   - 🌙 심야/새벽: 0-5시 (매우 낮은 활동)
   - 📅 주말: 브런치 문화 반영, 저녁 시간 분산


In [7]:
# ===================================================================
# 7️⃣ 올바른 스키마 이벤트 로그 분석 및 검증 함수
# ===================================================================

def validate_correct_schema(df):
    """올바른 스키마에 맞는지 검증하고 샘플 데이터 표시"""
    
    print("🔍 올바른 스키마 검증 및 샘플 데이터 확인")
    print("=" * 70)
    
    # 필수 컬럼 검증
    required_columns = [
        'event_name',        # String, No
        'event_id',          # String (UUID), No  
        'user_id',           # String, Yes
        'anonymous_id',      # String, No
        'session_id',        # String, No
        'context',           # JSON Object, No
        'event_properties',  # JSON Object, Yes
        'timestamp'          # Datetime (ISO 8601), No
    ]
    
    print("📋 필수 컬럼 검증:")
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"❌ 누락된 컬럼: {missing_columns}")
        return False
    else:
        print("✅ 모든 필수 컬럼 존재")
    
    # 데이터 타입 검증
    print(f"\n📊 데이터 타입 검증:")
    for col in df.columns:
        dtype = df[col].dtype
        print(f"   {col}: {dtype}")
    
    # 샘플 데이터 검증
    print(f"\n🔍 첫 번째 이벤트 샘플 데이터:")
    first_row = df.iloc[0]
    
    for col in required_columns:
        value = first_row[col]
        if col in ['context', 'event_properties']:
            try:
                parsed = json.loads(value)
                print(f"   {col}: {json.dumps(parsed, indent=4, ensure_ascii=False)}")
            except:
                print(f"   {col}: [JSON 파싱 오류] {value}")
        else:
            print(f"   {col}: {value}")
    
    # context 구조 검증
    print(f"\n📱 context 구조 검증:")
    try:
        context_sample = json.loads(df['context'].iloc[0])
        
        # context.page 검증
        if 'page' in context_sample:
            page_obj = context_sample['page']
            required_page_fields = ['name', 'url', 'path']
            missing_page_fields = [field for field in required_page_fields if field not in page_obj]
            
            if missing_page_fields:
                print(f"❌ context.page 누락 필드: {missing_page_fields}")
            else:
                print("✅ context.page 구조 정상")
                print(f"   - name: {page_obj['name']}")
                print(f"   - url: {page_obj['url']}")
                print(f"   - path: {page_obj['path']}")
        else:
            print("❌ context.page 누락")
        
        # context.user_segment 검증
        if 'user_segment' in context_sample:
            print(f"✅ context.user_segment 존재: {context_sample['user_segment']}")
        else:
            print("❌ context.user_segment 누락")
            
    except Exception as e:
        print(f"❌ context JSON 파싱 오류: {e}")
    
    # event_properties 샘플 검증
    print(f"\n🎯 event_properties 샘플 검증:")
    event_name_samples = df['event_name'].value_counts().head(5)
    
    for event_name, count in event_name_samples.items():
        print(f"\n   📌 {event_name} 이벤트 샘플:")
        event_sample = df[df['event_name'] == event_name].iloc[0]
        
        try:
            properties = json.loads(event_sample['event_properties'])
            print(f"      속성: {json.dumps(properties, indent=6, ensure_ascii=False)}")
        except:
            print(f"      속성 파싱 오류: {event_sample['event_properties']}")
    
    # 기본 통계
    print(f"\n📈 기본 통계:")
    print(f"   총 이벤트 수: {len(df):,}개")
    print(f"   고유 사용자 수: {df['user_id'].nunique():,}명")
    print(f"   고유 세션 수: {df['session_id'].nunique():,}개")
    print(f"   이벤트 타입 수: {df['event_name'].nunique()}개")
    
    print(f"\n🎯 이벤트 타입별 분포:")
    event_counts = df['event_name'].value_counts()
    for event_name, count in event_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {event_name}: {count:,}개 ({percentage:.1f}%)")
    
    return True

def analyze_hourly_distribution(df):
    """시간대별 이벤트 분포 상세 분석"""
    
    print("🕐 시간대별 이벤트 분포 분석")
    print("=" * 50)
    
    # timestamp를 datetime으로 변환
    df_copy = df.copy()
    df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'])
    df_copy['hour'] = df_copy['timestamp'].dt.hour
    df_copy['day_of_week'] = df_copy['timestamp'].dt.day_name()
    
    # 전체 시간대별 분포
    print("📊 전체 시간대별 이벤트 분포:")
    hourly_counts = df_copy['hour'].value_counts().sort_index()
    total_events = len(df_copy)
    
    for hour in range(24):
        count = hourly_counts.get(hour, 0)
        percentage = (count / total_events) * 100
        
        # 시간대별 특성 표시
        if hour in [18, 19, 20]:
            icon = "🔥🔥"  # 저녁 피크
        elif hour in [11, 12, 13]:
            icon = "🍽️"   # 점심 시간
        elif hour in [6, 7, 8, 9]:
            icon = "🌅"   # 아침 시간
        elif hour in [21, 22, 23]:
            icon = "🌃"   # 저녁 늦은 시간
        elif hour in [0, 1, 2, 3, 4, 5]:
            icon = "🌙"   # 심야/새벽
        else:
            icon = "📱"   # 일반 시간
        
        print(f"   {hour:02d}시: {count:,}개 ({percentage:.1f}%) {icon}")
    
    # 피크 시간대 분석
    print(f"\n🔥 피크 시간대 분석:")
    top_5_hours = hourly_counts.nlargest(5)
    for hour, count in top_5_hours.items():
        percentage = (count / total_events) * 100
        print(f"   {hour:02d}시: {count:,}개 ({percentage:.1f}%)")
    
    # 최저 시간대 분석
    print(f"\n🌙 최저 활동 시간대:")
    bottom_5_hours = hourly_counts.nsmallest(5)
    for hour, count in bottom_5_hours.items():
        percentage = (count / total_events) * 100
        print(f"   {hour:02d}시: {count:,}개 ({percentage:.1f}%)")
    
    # 요일별 시간대 패턴 (주말 vs 평일)
    print(f"\n📅 요일별 패턴 분석:")
    
    # 평일 (월-금)
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    weekday_data = df_copy[df_copy['day_of_week'].isin(weekdays)]
    
    if len(weekday_data) > 0:
        weekday_hourly = weekday_data['hour'].value_counts().sort_index()
        weekday_total = len(weekday_data)
        
        print(f"\n   📊 평일 피크 시간대 TOP 3:")
        for hour, count in weekday_hourly.nlargest(3).items():
            percentage = (count / weekday_total) * 100
            print(f"     {hour:02d}시: {count:,}개 ({percentage:.1f}%)")
    
    # 주말 (토-일)
    weekends = ['Saturday', 'Sunday']
    weekend_data = df_copy[df_copy['day_of_week'].isin(weekends)]
    
    if len(weekend_data) > 0:
        weekend_hourly = weekend_data['hour'].value_counts().sort_index()
        weekend_total = len(weekend_data)
        
        print(f"\n   📊 주말 피크 시간대 TOP 3:")
        for hour, count in weekend_hourly.nlargest(3).items():
            percentage = (count / weekend_total) * 100
            print(f"     {hour:02d}시: {count:,}개 ({percentage:.1f}%)")
    
    # 현실성 체크
    print(f"\n✅ 현실성 체크:")
    
    # 저녁 피크 체크 (17-20시)
    evening_peak = hourly_counts[17:21].sum()
    evening_percentage = (evening_peak / total_events) * 100
    print(f"   저녁 피크 (17-20시): {evening_peak:,}개 ({evening_percentage:.1f}%)")
    
    # 점심 시간 체크 (11-13시)
    lunch_time = hourly_counts[11:14].sum()
    lunch_percentage = (lunch_time / total_events) * 100
    print(f"   점심 시간 (11-13시): {lunch_time:,}개 ({lunch_percentage:.1f}%)")
    
    # 심야 시간 체크 (0-5시)
    midnight = hourly_counts[0:6].sum()
    midnight_percentage = (midnight / total_events) * 100
    print(f"   심야 시간 (0-5시): {midnight:,}개 ({midnight_percentage:.1f}%)")
    
    # 현실성 평가
    if evening_percentage > lunch_percentage and evening_percentage > midnight_percentage:
        print(f"   ✅ 저녁 피크가 가장 높음 - 현실적!")
    else:
        print(f"   ⚠️ 시간대 분포 재검토 필요")
    
    if midnight_percentage < 5:
        print(f"   ✅ 심야 시간 활동 적절히 낮음")
    else:
        print(f"   ⚠️ 심야 시간 활동이 너무 높음")
    
    return df_copy

def convert_to_csv_with_estimation(file_name='event_logs_10m.parquet'):
    """올바른 스키마 Parquet 파일을 CSV로 변환하고 크기 추정"""
    
    print("🔄 올바른 스키마 CSV 변환 및 크기 추정")
    print("=" * 50)
    
    # Parquet 파일 로드
    parquet_file = os.path.join(OUTPUT_LOG_PATH, file_name)
    
    if not os.path.exists(parquet_file):
        print(f"❌ Parquet 파일이 존재하지 않습니다: {parquet_file}")
        return
    
    # 파일 정보
    parquet_size_mb = os.path.getsize(parquet_file) / (1024 * 1024)
    print(f"📁 Parquet 파일 크기: {parquet_size_mb:.2f} MB")
    
    # 샘플 데이터로 크기 추정
    print("\n📊 CSV 크기 추정 중...")
    
    df = pd.read_parquet(parquet_file)
    
    # 전체 데이터 샘플링
    sample_size = min(10000, len(df))
    sample_df = df.sample(n=sample_size, random_state=42)
    
    # 메모리 기반 CSV 크기 측정
    import io
    csv_buffer = io.StringIO()
    sample_df.to_csv(csv_buffer, index=False)
    csv_sample_size = len(csv_buffer.getvalue().encode('utf-8'))
    
    # 전체 크기 추정
    estimated_csv_size_bytes = (csv_sample_size / sample_size) * len(df)
    estimated_csv_size_mb = estimated_csv_size_bytes / (1024 * 1024)
    estimated_csv_size_gb = estimated_csv_size_mb / 1024
    
    print(f"   샘플 데이터: {sample_size:,}행")
    print(f"   샘플 CSV 크기: {csv_sample_size / 1024:.2f} KB")
    print(f"   전체 예상 CSV 크기: {estimated_csv_size_mb:.2f} MB ({estimated_csv_size_gb:.2f} GB)")
    
    # 컬럼별 크기 분석
    print("\n📋 컬럼별 크기 분석:")
    for col in sample_df.columns:
        col_series = sample_df[col].astype(str)
        avg_char_count = col_series.str.len().mean()
        print(f"   {col}: 평균 {avg_char_count:.1f}자")
    
    # 압축률 분석
    compression_ratio = estimated_csv_size_mb / parquet_size_mb
    print(f"\n📈 압축률 분석:")
    print(f"   Parquet → CSV 확장 비율: {compression_ratio:.1f}x")
    print(f"   Parquet 압축 효율: {(1 - 1/compression_ratio)*100:.1f}%")
    
    # 사용자 선택
    if estimated_csv_size_gb > 1.0:
        print(f"\n⚠️  주의: 예상 CSV 크기가 {estimated_csv_size_gb:.2f} GB입니다!")
        user_input = input("변환을 계속하시겠습니까? (y/n): ")
        if user_input.lower() != 'y':
            print("CSV 변환이 취소되었습니다.")
            return
    
    # CSV 변환 실행
    print(f"\n🔄 CSV 변환 시작...")
    csv_file = os.path.join(OUTPUT_LOG_PATH, 'event_logs_10m.csv')
    
    try:
        # 청크 단위로 변환 (메모리 효율성)
        chunk_size = 50000
        first_chunk = True
        
        total_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
        print(f"   총 {total_chunks}개 청크로 분할 처리")
        
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            
            # 첫 번째 청크는 헤더 포함, 나머지는 헤더 없이
            mode = 'w' if first_chunk else 'a'
            header = first_chunk
            
            chunk.to_csv(csv_file, mode=mode, header=header, index=False)
            
            current_chunk = (i // chunk_size) + 1
            print(f"   처리 완료: {current_chunk}/{total_chunks} 청크")
            
            first_chunk = False
        
        # 최종 크기 확인
        actual_csv_size_mb = os.path.getsize(csv_file) / (1024 * 1024)
        actual_csv_size_gb = actual_csv_size_mb / 1024
        
        print("=" * 50)
        print("✅ CSV 변환 완료!")
        print("=" * 50)
        print(f"📁 CSV 파일: {csv_file}")
        print(f"💽 실제 크기: {actual_csv_size_mb:.2f} MB ({actual_csv_size_gb:.2f} GB)")
        print(f"📊 예상 대비: {(actual_csv_size_mb/estimated_csv_size_mb)*100:.1f}% 정확도")
        
    except Exception as e:
        print(f"❌ CSV 변환 오류: {e}")

print("✅ 올바른 스키마 분석 및 변환 함수 준비 완료 (시간대 분포 분석 포함)")

✅ 올바른 스키마 분석 및 변환 함수 준비 완료 (시간대 분포 분석 포함)


In [8]:
# ===================================================================
# 8️⃣ 올바른 스키마 이벤트 로그 생성 및 검증 실행
# ===================================================================

print("🚀 올바른 스키마로 이벤트 로그 생성 시작!")
print("=" * 60)

# 올바른 스키마로 이벤트 로그 생성
correct_event_df = generate_correct_schema_event_logs()

if correct_event_df is not None:
    print(f"\n✅ 올바른 스키마 이벤트 로그 생성 성공!")
    print(f"📂 파일 위치: {OUTPUT_LOG_PATH}event_logs_10m.parquet")
    
    # 올바른 스키마 검증 실행
    print(f"\n🔍 올바른 스키마 검증 시작!")
    schema_valid = validate_correct_schema(correct_event_df)
    
    if schema_valid:
        print(f"\n🎉 올바른 스키마 검증 완료!")
        print(f"📊 총 {len(correct_event_df):,}개의 올바른 스키마 이벤트가 생성되었습니다.")
        
        # 시간대별 분포 분석 실행
        print(f"\n" + "="*60)
        print("🕐 현실적인 시간대 분포 검증을 위한 상세 분석 시작!")
        print("="*60)
        
        analyzed_df = analyze_hourly_distribution(correct_event_df)
        
        print(f"\n🎯 시간대 분포 분석 완료!")
        print(f"📈 저녁 피크(17-20시), 점심(11-13시), 아침(6-9시) 패턴이 올바르게 적용되었는지 확인하세요.")
        
    else:
        print(f"\n❌ 스키마 검증 실패!")
        
else:
    print("❌ 올바른 스키마 이벤트 로그 생성 실패!")

🚀 올바른 스키마로 이벤트 로그 생성 시작!
🚀 올바른 스키마로 이벤트 로그 생성 시작!
👥 Step 1: 사용자 페르소나 할당...
🎭 사용자 페르소나 할당 시작...
✅ 페르소나 할당 완료: 30,000명

📊 Demographic Segment 분포:
   - FEMALE_40_PLUS: 35.8%
   - FEMALE_30S: 20.5%
   - MALE_40_PLUS: 14.4%
   - FEMALE_20S: 14.4%
   - MALE_30S: 8.7%
   - MALE_20S: 6.3%

⚡ 이용 강도 분포:
   - ACTIVE_USER: 54.7%
   - CASUAL_USER: 30.0%
   - POWER_USER: 15.3%

🍳 요리 스타일 분포:
   - HEALTHY_CONSCIOUS: 25.4%
   - COMFORT_FOOD: 24.9%
   - DESSERT_FOCUSED: 20.1%
   - QUICK_CONVENIENT: 19.6%
   - DIVERSE_EXPLORER: 10.0%
📂 Step 2: 데이터 준비...
   사용자: 30,000명
   레시피: 50,000개 (샘플링)
   프로필: 30,000개

📅 Step 3: 시뮬레이션 일정 생성...
   총 357일 시뮬레이션
   시작 DAU: 108명
   종료 DAU: 1,996명

👥 Step 4: 사용자 배치 생성...
   사용자: 30,000명
   레시피: 50,000개 (샘플링)
   프로필: 30,000개

📅 Step 3: 시뮬레이션 일정 생성...
   총 357일 시뮬레이션
   시작 DAU: 108명
   종료 DAU: 1,996명

👥 Step 4: 사용자 배치 생성...
   월별 사용자 배치: 12개   월별 사용자 배치: 12개

🔄 Step 5: 이벤트 생성 시작...
🕐 현실적인 시간대 분포 적용 - 저녁 피크(17-20시), 점심(11-13시), 아침(6-9시)

   📆 1월 처리 중... (31일)


🔄 Step 5: 이벤

In [9]:
# ===================================================================
# 9️⃣ 카프카 전송용 샘플 데이터 생성
# ===================================================================

def generate_kafka_sample_data(original_df, sample_ratios=[0.1, 0.02, 0.01]):
    """
    카프카 전송용 샘플 데이터 생성
    
    Args:
        original_df: 원본 이벤트 로그 데이터프레임
        sample_ratios: 샘플링 비율 리스트 [1/10, 1/50, 1/100]
    """
    
    print("📤 카프카 전송용 샘플 데이터 생성 시작!")
    print("=" * 60)
    
    if original_df is None or len(original_df) == 0:
        print("❌ 원본 데이터가 없습니다!")
        return
    
    original_count = len(original_df)
    print(f"📊 원본 데이터: {original_count:,}개 이벤트")
    
    # 카프카 샘플 데이터 저장 디렉토리
    kafka_sample_path = os.path.join(OUTPUT_LOG_PATH, 'kafka_samples')
    os.makedirs(kafka_sample_path, exist_ok=True)
    
    # 각 비율별로 샘플 생성
    sample_info = {
        0.1: {'name': '1_10', 'description': '1/10 샘플 (10%)'},
        0.02: {'name': '1_50', 'description': '1/50 샘플 (2%)'},
        0.01: {'name': '1_100', 'description': '1/100 샘플 (1%)'}
    }
    
    generated_samples = {}
    
    for ratio in sample_ratios:
        if ratio not in sample_info:
            continue
            
        sample_name = sample_info[ratio]['name']
        sample_desc = sample_info[ratio]['description']
        
        print(f"\n🔄 {sample_desc} 생성 중...")
        
        # 샘플링 (시간순 유지를 위해 정렬 후 균등 샘플링)
        df_sorted = original_df.sort_values('timestamp').reset_index(drop=True)
        
        # 균등 간격으로 샘플링
        step = int(1 / ratio)
        sample_indices = range(0, len(df_sorted), step)
        sample_df = df_sorted.iloc[sample_indices].copy()
        
        sample_count = len(sample_df)
        
        print(f"   📏 샘플링 비율: {ratio:.1%}")
        print(f"   📊 샘플 크기: {sample_count:,}개 이벤트")
        print(f"   ⏰ 시간 범위: {sample_df['timestamp'].min()} ~ {sample_df['timestamp'].max()}")
        
        # 파일 저장
        parquet_file = os.path.join(kafka_sample_path, f'kafka_sample_{sample_name}.parquet')
        json_file = os.path.join(kafka_sample_path, f'kafka_sample_{sample_name}.jsonl')
        
        # Parquet 저장
        sample_df.to_parquet(parquet_file, index=False)
        parquet_size_mb = os.path.getsize(parquet_file) / (1024 * 1024)
        
        # JSONL 저장 (카프카에서 자주 사용)
        print(f"   💾 JSONL 포맷으로 저장 중...")
        with open(json_file, 'w', encoding='utf-8') as f:
            for _, row in sample_df.iterrows():
                # 각 행을 JSON으로 변환
                json_record = {
                    'event_name': row['event_name'],
                    'event_id': row['event_id'],
                    'user_id': row['user_id'],
                    'anonymous_id': row['anonymous_id'],
                    'session_id': row['session_id'],
                    'context': json.loads(row['context']),  # JSON 문자열을 객체로 변환
                    'event_properties': json.loads(row['event_properties']),  # JSON 문자열을 객체로 변환
                    'timestamp': row['timestamp']
                }
                f.write(json.dumps(json_record, ensure_ascii=False) + '\n')
        
        jsonl_size_mb = os.path.getsize(json_file) / (1024 * 1024)
        
        print(f"   ✅ 저장 완료:")
        print(f"      📁 Parquet: {parquet_file} ({parquet_size_mb:.2f} MB)")
        print(f"      📄 JSONL: {json_file} ({jsonl_size_mb:.2f} MB)")
        
        # 샘플 통계
        print(f"   📈 샘플 통계:")
        print(f"      고유 사용자: {sample_df['user_id'].nunique():,}명")
        print(f"      고유 세션: {sample_df['session_id'].nunique():,}개")
        print(f"      이벤트 타입: {sample_df['event_name'].nunique()}개")
        
        # 시간대별 분포 확인
        sample_df_copy = sample_df.copy()
        sample_df_copy['timestamp'] = pd.to_datetime(sample_df_copy['timestamp'])
        sample_df_copy['hour'] = sample_df_copy['timestamp'].dt.hour
        
        hourly_dist = sample_df_copy['hour'].value_counts().sort_index()
        peak_hours = hourly_dist.nlargest(3)
        
        print(f"      피크 시간대:")
        for hour, count in peak_hours.items():
            percentage = (count / len(sample_df_copy)) * 100
            print(f"        {hour:02d}시: {count:,}개 ({percentage:.1f}%)")
        
        generated_samples[sample_name] = {
            'dataframe': sample_df,
            'parquet_file': parquet_file,
            'jsonl_file': json_file,
            'count': sample_count,
            'ratio': ratio
        }
    
    # 카프카 프로듀서 예시 코드 생성
    producer_code_file = os.path.join(kafka_sample_path, 'kafka_producer_example.py')
    
    producer_code = '''#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
카프카 프로듀서 예시 코드
생성된 JSONL 샘플 데이터를 카프카로 전송하는 예시
"""

import json
import time
from kafka import KafkaProducer
from datetime import datetime

def create_kafka_producer(bootstrap_servers=['localhost:9092']):
    """카프카 프로듀서 생성"""
    return KafkaProducer(
        bootstrap_servers=bootstrap_servers,
        value_serializer=lambda v: json.dumps(v, ensure_ascii=False).encode('utf-8'),
        key_serializer=lambda k: k.encode('utf-8') if k else None,
        acks='all',  # 모든 복제본에서 확인
        retries=3,
        batch_size=16384,
        linger_ms=10,
        buffer_memory=33554432
    )

def send_jsonl_to_kafka(jsonl_file, topic_name, producer, delay_ms=100):
    """JSONL 파일을 카프카 토픽으로 전송"""
    
    print(f"📤 카프카 전송 시작: {jsonl_file} -> {topic_name}")
    
    sent_count = 0
    error_count = 0
    
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                # JSON 파싱
                event_data = json.loads(line.strip())
                
                # 키 생성 (선택적)
                key = f"{event_data['user_id']}_{event_data['session_id']}"
                
                # 카프카로 전송
                future = producer.send(
                    topic=topic_name,
                    key=key,
                    value=event_data,
                    timestamp_ms=int(time.time() * 1000)
                )
                
                # 전송 결과 확인 (비동기)
                future.add_callback(lambda metadata: None)
                future.add_errback(lambda e: print(f"❌ 전송 실패: {e}"))
                
                sent_count += 1
                
                # 진행률 표시
                if sent_count % 1000 == 0:
                    print(f"   📊 진행률: {sent_count:,}개 전송됨")
                
                # 전송 간격 조절
                if delay_ms > 0:
                    time.sleep(delay_ms / 1000.0)
                
            except Exception as e:
                error_count += 1
                print(f"❌ 라인 {line_num} 처리 오류: {e}")
                continue
    
    # 모든 메시지 전송 완료 대기
    producer.flush()
    
    print(f"✅ 카프카 전송 완료!")
    print(f"   📊 성공: {sent_count:,}개")
    print(f"   ❌ 실패: {error_count:,}개")

# 사용 예시
if __name__ == "__main__":
    
    # 카프카 설정
    KAFKA_SERVERS = ['localhost:9092']
    TOPIC_NAME = 'recipe-events'
    
    # 샘플 파일 경로
    SAMPLE_FILES = {
        '1/10 샘플': 'kafka_sample_1_10.jsonl',
        '1/50 샘플': 'kafka_sample_1_50.jsonl', 
        '1/100 샘플': 'kafka_sample_1_100.jsonl'
    }
    
    try:
        # 프로듀서 생성
        producer = create_kafka_producer(KAFKA_SERVERS)
        
        # 원하는 샘플 파일 선택 (예: 1/100 샘플)
        selected_file = SAMPLE_FILES['1/100 샘플']
        
        # 카프카로 전송
        send_jsonl_to_kafka(
            jsonl_file=selected_file,
            topic_name=TOPIC_NAME,
            producer=producer,
            delay_ms=50  # 50ms 간격 (초당 20개)
        )
        
    except Exception as e:
        print(f"❌ 카프카 전송 중 오류: {e}")
        
    finally:
        if 'producer' in locals():
            producer.close()
'''

    with open(producer_code_file, 'w', encoding='utf-8') as f:
        f.write(producer_code)
    
    # 요약 정보 출력
    print("\n" + "=" * 60)
    print("🎉 카프카 샘플 데이터 생성 완료!")
    print("=" * 60)
    
    print(f"📁 저장 위치: {kafka_sample_path}")
    
    for sample_name, info in generated_samples.items():
        ratio_desc = sample_info[info['ratio']]['description']
        print(f"\n📦 {ratio_desc}:")
        print(f"   📊 이벤트 수: {info['count']:,}개")
        print(f"   📁 Parquet: kafka_sample_{sample_name}.parquet")
        print(f"   📄 JSONL: kafka_sample_{sample_name}.jsonl")
    
    print(f"\n🔧 카프카 프로듀서 예시 코드: kafka_producer_example.py")
    
    print(f"\n💡 사용 가이드:")
    print(f"   1. JSONL 파일은 카프카 프로듀서에서 직접 사용 가능")
    print(f"   2. Parquet 파일은 판다스로 읽어서 분석 가능")
    print(f"   3. kafka_producer_example.py를 참고하여 카프카 전송")
    
    print(f"\n📋 카프카 토픽 권장 설정:")
    print(f"   - 토픽명: recipe-events")
    print(f"   - 파티션: 3-6개 (데이터 볼륨에 따라)")
    print(f"   - 복제본: 2-3개 (안정성)")
    print(f"   - 압축: gzip 또는 snappy")
    
    return generated_samples

# 카프카 샘플 데이터 생성 실행
if 'correct_event_df' in locals() and correct_event_df is not None:
    print("\n" + "🚀" * 20)
    kafka_samples = generate_kafka_sample_data(
        original_df=correct_event_df,
        sample_ratios=[0.1, 0.02, 0.01]  # 1/10, 1/50, 1/100
    )
else:
    print("⚠️ 먼저 이벤트 로그를 생성한 후 카프카 샘플을 만들어주세요.")

# CSV 변환이 필요한 경우 아래 주석을 해제하고 실행
# convert_to_csv_with_estimation('event_logs_10m.parquet')

print("\n✅ 모든 올바른 스키마 이벤트 로그 생성 및 검증 완료!")
print("🎯 필요시 위 주석을 해제하여 CSV 변환을 실행하세요.")
print("📤 카프카 샘플 데이터도 함께 생성되었습니다!")


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
📤 카프카 전송용 샘플 데이터 생성 시작!
📊 원본 데이터: 10,031,152개 이벤트

🔄 1/10 샘플 (10%) 생성 중...
   📏 샘플링 비율: 10.0%
   📊 샘플 크기: 1,003,116개 이벤트
   📏 샘플링 비율: 10.0%
   📊 샘플 크기: 1,003,116개 이벤트
   ⏰ 시간 범위: 2024-08-01T00:09:24.000000Z ~ 2025-07-24T00:29:42.000000Z
   ⏰ 시간 범위: 2024-08-01T00:09:24.000000Z ~ 2025-07-24T00:29:42.000000Z
   💾 JSONL 포맷으로 저장 중...
   💾 JSONL 포맷으로 저장 중...
   ✅ 저장 완료:
      📁 Parquet: event_logs/kafka_samples\kafka_sample_1_10.parquet (114.76 MB)
      📄 JSONL: event_logs/kafka_samples\kafka_sample_1_10.jsonl (450.36 MB)
   📈 샘플 통계:
      고유 사용자: 11,844명
   ✅ 저장 완료:
      📁 Parquet: event_logs/kafka_samples\kafka_sample_1_10.parquet (114.76 MB)
      📄 JSONL: event_logs/kafka_samples\kafka_sample_1_10.jsonl (450.36 MB)
   📈 샘플 통계:
      고유 사용자: 11,844명
      고유 세션: 478,109개
      이벤트 타입: 11개
      고유 세션: 478,109개
      이벤트 타입: 11개
      피크 시간대:
        19시: 123,608개 (12.3%)
        18시: 123,311개 (12.3%)
        20시: 94,896개 (9.5%)

🔄 1/50 샘플 (2%) 생성 중...
      피크 시간대:

In [10]:
# ===================================================================
# 🔟 이벤트 타입별 샘플 데이터 표시 (Nested Dictionary 형태)
# ===================================================================

def display_event_samples(df, max_samples_per_event=1):
    """
    각 event_name별로 샘플 데이터를 nested dictionary 형태로 표시
    
    Args:
        df: 이벤트 로그 데이터프레임
        max_samples_per_event: 각 이벤트 타입별로 표시할 최대 샘플 수
    """
    
    print("📋 이벤트 타입별 샘플 데이터 (Nested Dictionary 형태)")
    print("=" * 70)
    
    if df is None or len(df) == 0:
        print("❌ 표시할 데이터가 없습니다!")
        return
    
    # 모든 고유 이벤트 타입 가져오기
    event_types = df['event_name'].unique()
    event_counts = df['event_name'].value_counts()
    
    print(f"📊 총 {len(event_types)}개의 이벤트 타입 발견:")
    for event_type in sorted(event_types):
        count = event_counts[event_type]
        percentage = (count / len(df)) * 100
        print(f"   - {event_type}: {count:,}개 ({percentage:.1f}%)")
    
    print(f"\n" + "="*70)
    print(f"🔍 각 이벤트 타입별 샘플 데이터 (각 {max_samples_per_event}개씩)")
    print("="*70)
    
    # 전체 샘플 데이터를 담을 nested dictionary
    samples_dict = {}
    
    for event_type in sorted(event_types):
        print(f"\n🎯 [{event_type}] 이벤트 샘플:")
        print("-" * 50)
        
        # 해당 이벤트 타입의 샘플 가져오기
        event_samples = df[df['event_name'] == event_type].head(max_samples_per_event)
        
        # 이벤트 타입별 샘플 리스트 초기화
        samples_dict[event_type] = []
        
        for idx, (_, row) in enumerate(event_samples.iterrows(), 1):
            
            # nested dictionary 구조로 데이터 구성
            sample_data = {
                'event_name': row['event_name'],
                'event_id': row['event_id'],
                'user_id': row['user_id'],
                'anonymous_id': row['anonymous_id'],
                'session_id': row['session_id'],
                'timestamp': row['timestamp']
            }
            
            # Context 파싱
            try:
                context = json.loads(row['context'])
                sample_data['context'] = context
            except Exception as e:
                sample_data['context'] = {'parsing_error': str(e)}
            
            # Event Properties 파싱
            try:
                properties = json.loads(row['event_properties'])
                sample_data['event_properties'] = properties
            except Exception as e:
                sample_data['event_properties'] = {'parsing_error': str(e)}
            
            # 샘플 데이터를 리스트에 추가
            samples_dict[event_type].append(sample_data)
            
            # Pretty print로 nested dictionary 출력
            print(f"\n📝 샘플 #{idx}:")
            import pprint
            pp = pprint.PrettyPrinter(indent=2, width=80, depth=None)
            pp.pprint(sample_data)
            
            if idx < len(event_samples):
                print("   " + "·" * 40)
    
    return samples_dict
    
    print(f"\n" + "="*70)
    print("✅ 이벤트 타입별 샘플 데이터 표시 완료!")
    print("="*70)

def display_event_summary_table(df):
    """이벤트 타입별 요약 테이블 표시"""
    
    if df is None or len(df) == 0:
        print("❌ 요약할 데이터가 없습니다!")
        return
    
    print("\n📊 이벤트 타입별 요약 테이블")
    print("=" * 80)
    
    # 이벤트별 통계 계산
    event_stats = []
    
    for event_type in sorted(df['event_name'].unique()):
        event_data = df[df['event_name'] == event_type]
        
        # 기본 통계
        count = len(event_data)
        percentage = (count / len(df)) * 100
        unique_users = event_data['user_id'].nunique()
        unique_sessions = event_data['session_id'].nunique()
        
        # 시간 범위
        timestamps = pd.to_datetime(event_data['timestamp'])
        time_range = f"{timestamps.min().strftime('%m-%d %H:%M')} ~ {timestamps.max().strftime('%m-%d %H:%M')}"
        
        # 샘플 properties 키
        try:
            sample_properties = json.loads(event_data['event_properties'].iloc[0])
            property_keys = list(sample_properties.keys()) if sample_properties else []
            property_preview = ", ".join(property_keys[:3])
            if len(property_keys) > 3:
                property_preview += f", ... (총 {len(property_keys)}개)"
        except:
            property_preview = "파싱 오류"
        
        event_stats.append({
            'Event Type': event_type,
            'Count': f"{count:,}",
            'Percentage': f"{percentage:.1f}%",
            'Users': f"{unique_users:,}",
            'Sessions': f"{unique_sessions:,}",
            'Time Range': time_range,
            'Properties': property_preview
        })
    
    # 테이블 형태로 출력
    if event_stats:
        # 헤더
        headers = ['Event Type', 'Count', '%', 'Users', 'Sessions', 'Properties']
        
        print(f"{'Event Type':<25} {'Count':<10} {'%':<6} {'Users':<8} {'Sessions':<10} {'Properties'}")
        print("-" * 80)
        
        for stat in event_stats:
            print(f"{stat['Event Type']:<25} {stat['Count']:<10} {stat['Percentage']:<6} "
                  f"{stat['Users']:<8} {stat['Sessions']:<10} {stat['Properties'][:25]}")
    
    print("=" * 80)

# 생성된 이벤트 로그의 샘플 데이터 표시
if 'correct_event_df' in locals() and correct_event_df is not None:
    print("\n" + "🔍" * 20)
    print("이벤트 타입별 상세 샘플 데이터를 Nested Dictionary 형태로 표시합니다!")
    print("🔍" * 20)
    
    # 요약 테이블 먼저 표시
    display_event_summary_table(correct_event_df)
    
    # 각 이벤트별 상세 샘플 표시 (nested dictionary 반환)
    samples_dict = display_event_samples(correct_event_df, max_samples_per_event=1)
    
    print(f"\n" + "="*70)
    print("✅ 이벤트 타입별 샘플 데이터 표시 완료!")
    print("="*70)
    
    print(f"\n💡 반환된 데이터 구조:")
    print(f"   - samples_dict 변수에 모든 샘플이 nested dictionary 형태로 저장됨")
    print(f"   - 구조: samples_dict[event_name][sample_index] = {{...}}")
    print(f"   - 예시: samples_dict['view_page'][0]['context']['page']['name']")
    
    print(f"\n🔧 더 많은 샘플을 보려면:")
    print(f"   samples_dict = display_event_samples(correct_event_df, max_samples_per_event=2)")
    
    print(f"\n📋 특정 이벤트만 보려면:")
    print(f"   import pprint")
    print(f"   pprint.pprint(samples_dict['view_page'])  # view_page 이벤트만")
    
else:
    print("⚠️ 먼저 이벤트 로그를 생성한 후 샘플 데이터를 확인하세요.")


🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍
이벤트 타입별 상세 샘플 데이터를 Nested Dictionary 형태로 표시합니다!
🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍🔍

📊 이벤트 타입별 요약 테이블
Event Type                Count      %      Users    Sessions   Properties
--------------------------------------------------------------------------------
auth_success              584,787    5.8%   11,844   439,395    method, type
click_ads                 134,358    1.3%   11,486   117,821    ad_id, ad_type, target_ur
click_auth_button         355,312    3.5%   11,809   271,188    type
click_bookmark            443,220    4.4%   11,775   303,775    recipe_id, action
click_like                441,637    4.4%   11,785   306,144    recipe_id, action
click_recipe_from_list    1,786,735  17.8%  11,848   577,245    recipe_id, rank
create_comment            550,788    5.5%   11,807   350,399    recipe_id, comment_length
search_recipe             1,831,844  18.3%  11,851   593,840    search_type, search_keywo
view_ads                  355,090    3.5%   11,793   255,168    ad_id, ad_t

In [11]:
# ===================================================================
# 📏 event_logs_10m.parquet → CSV 크기 추정
# ===================================================================

def estimate_csv_size_from_parquet(parquet_file_name='event_logs_10m.parquet'):
    """
    Parquet 파일을 CSV로 변환했을 때의 크기를 추정
    
    Args:
        parquet_file_name: 추정할 parquet 파일명
    """
    
    print("📏 Parquet → CSV 크기 추정 시작!")
    print("=" * 60)
    
    parquet_file = os.path.join(OUTPUT_LOG_PATH, parquet_file_name)
    
    # 파일 존재 여부 확인
    if not os.path.exists(parquet_file):
        print(f"❌ 파일이 존재하지 않습니다: {parquet_file}")
        return
    
    # Parquet 파일 크기
    parquet_size_bytes = os.path.getsize(parquet_file)
    parquet_size_mb = parquet_size_bytes / (1024 * 1024)
    parquet_size_gb = parquet_size_mb / 1024
    
    print(f"📁 Parquet 파일 정보:")
    print(f"   파일명: {parquet_file_name}")
    print(f"   크기: {parquet_size_mb:.2f} MB ({parquet_size_gb:.3f} GB)")
    print(f"   바이트: {parquet_size_bytes:,} bytes")
    
    try:
        # 샘플 데이터로 크기 추정
        print(f"\n📊 CSV 크기 추정 중...")
        
        df = pd.read_parquet(parquet_file)
        total_rows = len(df)
        
        print(f"   총 데이터: {total_rows:,}행 × {len(df.columns)}열")
        
        # 샘플 크기 결정 (최대 50,000행)
        sample_size = min(50000, total_rows)
        sample_df = df.sample(n=sample_size, random_state=42)
        
        print(f"   샘플 크기: {sample_size:,}행으로 추정")
        
        # 메모리 기반 CSV 크기 측정
        import io
        csv_buffer = io.StringIO()
        sample_df.to_csv(csv_buffer, index=False)
        csv_sample_content = csv_buffer.getvalue()
        csv_sample_size_bytes = len(csv_sample_content.encode('utf-8'))
        
        # 전체 크기 추정
        estimated_csv_size_bytes = (csv_sample_size_bytes / sample_size) * total_rows
        estimated_csv_size_mb = estimated_csv_size_bytes / (1024 * 1024)
        estimated_csv_size_gb = estimated_csv_size_mb / 1024
        
        # 압축률 계산
        compression_ratio = estimated_csv_size_mb / parquet_size_mb
        compression_efficiency = (1 - parquet_size_mb / estimated_csv_size_mb) * 100
        
        print(f"\n📈 크기 추정 결과:")
        print(f"   샘플 CSV 크기: {csv_sample_size_bytes / 1024:.2f} KB ({sample_size:,}행)")
        print(f"   행당 평균 크기: {csv_sample_size_bytes / sample_size:.1f} bytes")
        
        print(f"\n🎯 전체 CSV 예상 크기:")
        print(f"   예상 크기: {estimated_csv_size_mb:.2f} MB")
        print(f"   예상 크기: {estimated_csv_size_gb:.3f} GB")
        print(f"   예상 바이트: {estimated_csv_size_bytes:,.0f} bytes")
        
        print(f"\n📊 압축률 분석:")
        print(f"   Parquet vs CSV: {compression_ratio:.1f}x 차이")
        print(f"   Parquet 압축 효율: {compression_efficiency:.1f}%")
        print(f"   CSV가 Parquet보다 {compression_ratio:.1f}배 더 큼")
        
        # 컬럼별 크기 분석
        print(f"\n📋 컬럼별 평균 크기 분석:")
        col_sizes = {}
        for col in sample_df.columns:
            col_series = sample_df[col].astype(str)
            avg_char_count = col_series.str.len().mean()
            col_sizes[col] = avg_char_count
            
        # 크기가 큰 컬럼 순으로 정렬
        sorted_cols = sorted(col_sizes.items(), key=lambda x: x[1], reverse=True)
        
        for col, avg_size in sorted_cols:
            print(f"   {col}: 평균 {avg_size:.1f}자")
        
        # 경고 메시지
        if estimated_csv_size_gb > 2.0:
            print(f"\n⚠️  주의사항:")
            print(f"   - 예상 CSV 크기가 {estimated_csv_size_gb:.1f} GB로 매우 큽니다!")
            print(f"   - 메모리 부족이나 로딩 시간이 오래 걸릴 수 있습니다")
            print(f"   - 청크 단위로 처리하거나 Parquet 사용을 권장합니다")
        elif estimated_csv_size_gb > 1.0:
            print(f"\n💡 권장사항:")
            print(f"   - 예상 CSV 크기가 {estimated_csv_size_gb:.1f} GB입니다")
            print(f"   - 대용량이므로 pandas.read_csv()에서 chunksize 사용 권장")
        else:
            print(f"\n✅ 적정 크기:")
            print(f"   - 예상 CSV 크기가 {estimated_csv_size_mb:.0f} MB로 적당합니다")
            print(f"   - 일반적인 방법으로 로딩 가능합니다")
        
        # 로딩 시간 추정
        print(f"\n⏱️  예상 로딩 시간:")
        # SSD 기준 대략적인 읽기 속도 (100MB/s로 가정)
        estimated_read_time = estimated_csv_size_mb / 100
        print(f"   CSV 파일 읽기: 약 {estimated_read_time:.1f}초 (SSD 기준)")
        print(f"   Parquet 파일 읽기: 약 {parquet_size_mb / 200:.1f}초 (더 빠름)")
        
        # 실제 변환 여부 선택
        print(f"\n🔧 실제 CSV 변환:")
        print(f"   변환을 원하면 다음 함수를 호출하세요:")
        print(f"   convert_to_csv_with_estimation('{parquet_file_name}')")
        
        return {
            'parquet_size_mb': parquet_size_mb,
            'estimated_csv_size_mb': estimated_csv_size_mb,
            'compression_ratio': compression_ratio,
            'total_rows': total_rows,
            'estimated_read_time': estimated_read_time
        }
        
    except Exception as e:
        print(f"❌ 크기 추정 중 오류 발생: {e}")
        return None

# event_logs_10m.parquet 파일 크기 추정 실행
print("🔍 생성된 이벤트 로그 파일의 CSV 변환 크기를 추정합니다!")
print("=" * 70)

size_estimation = estimate_csv_size_from_parquet('event_logs_10m.parquet')

if size_estimation:
    print(f"\n📊 요약:")
    print(f"   📁 Parquet: {size_estimation['parquet_size_mb']:.2f} MB")
    print(f"   📄 예상 CSV: {size_estimation['estimated_csv_size_mb']:.2f} MB")
    print(f"   🗜️  압축률: {size_estimation['compression_ratio']:.1f}x")
    print(f"   📝 총 데이터: {size_estimation['total_rows']:,}행")
    
print(f"\n💡 참고:")
print(f"   - Parquet은 컬럼형 압축으로 저장 공간 효율적")
print(f"   - CSV는 텍스트 형태로 호환성이 좋지만 용량이 큼")
print(f"   - 분석용도라면 Parquet 사용 권장")
print(f"   - 다른 시스템 연동시에만 CSV 변환 고려")

🔍 생성된 이벤트 로그 파일의 CSV 변환 크기를 추정합니다!
📏 Parquet → CSV 크기 추정 시작!
📁 Parquet 파일 정보:
   파일명: event_logs_10m.parquet
   크기: 970.72 MB (0.948 GB)
   바이트: 1,017,875,899 bytes

📊 CSV 크기 추정 중...
   총 데이터: 10,031,152행 × 8열
   총 데이터: 10,031,152행 × 8열
   샘플 크기: 50,000행으로 추정
   샘플 크기: 50,000행으로 추정

📈 크기 추정 결과:
   샘플 CSV 크기: 18084.73 KB (50,000행)
   행당 평균 크기: 370.4 bytes

🎯 전체 CSV 예상 크기:
   예상 크기: 3543.18 MB
   예상 크기: 3.460 GB
   예상 바이트: 3,715,290,129 bytes

📊 압축률 분석:
   Parquet vs CSV: 3.7x 차이
   Parquet 압축 효율: 72.6%
   CSV가 Parquet보다 3.7배 더 큼

📋 컬럼별 평균 크기 분석:
   context: 평균 114.9자
   event_properties: 평균 60.4자
   event_id: 평균 36.0자
   anonymous_id: 평균 36.0자
   session_id: 평균 36.0자
   timestamp: 평균 27.0자
   event_name: 평균 14.2자
   user_id: 평균 4.6자

⚠️  주의사항:
   - 예상 CSV 크기가 3.5 GB로 매우 큽니다!
   - 메모리 부족이나 로딩 시간이 오래 걸릴 수 있습니다
   - 청크 단위로 처리하거나 Parquet 사용을 권장합니다

⏱️  예상 로딩 시간:
   CSV 파일 읽기: 약 35.4초 (SSD 기준)
   Parquet 파일 읽기: 약 4.9초 (더 빠름)

🔧 실제 CSV 변환:
   변환을 원하면 다음 함수를 호출하세요:
   convert_to_csv_with_estim